In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


import statsmodels.api as sm


In [111]:
class aviation_data_model:
    def __init__(self, x, y, drop_categories = None, test_size = 0.3, random_state = 2023):
        self.x = x.copy()
        self.drop_categories = drop_categories
        self.y = y
        self.random_state = random_state
        self.test_size = test_size
        self.model = None
        self.x_oh = None
        self.x_con = None
        self.results = None
        self.pvalues = None

        
    def one_hot(self):
        #Check to see if dummy parameter passed correctly
        if type(self.drop_categories[0]) == tuple:
            
            categories = []
            for i, j in self.drop_categories:
                # Collect subset of columns to dummify
                categories.append(i)
                
                # If a specific drop value has been given for a column, add a 0- to it so it is 'first' and drops accordingly
                if j != 'first':
                    self.x[i] = np.where(self.x[i] == j, '0-' + j, self.x[i])
        else:
            raise TypeError('Invalid OneHotEncoder values. Use None or a list of tuples of (column name, category name')
            
        self.x_oh = pd.get_dummies(data= self.x, columns = categories, drop_first=True)
        # Log line - uncomment below to check shapes after important steps
        # print(f"OneHot Resulting Shape: {self.x_oh.shape}")
        return self.x_oh
    
    def train_test(self, x, y):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=self.test_size, random_state=self.random_state)
        return (X_train, X_test, y_train, y_test)
        
    
    def fit_binomial(self):
        # Start with OneHotEncoding if given
        if self.drop_categories:
            oh = self.one_hot()
            
        # Add column for constant per statsmodels GLM requirements
        x_con = sm.add_constant(oh[list(oh.columns)])
        self.x_con = x_con
        
        # Log line - uncomment below to check shapes after important steps
        # print(f"After constant Resulting Shape: {self.x_con.shape}")
        
        #Train Test Split
        X_train, X_test, y_train, y_test = self.train_test(x_con, self.y)

        
        # Log lines - uncomment below to check shapes after important steps
        # print(f"After tts X_train Resulting Shape: {X_train.shape}")
        # print(f"After tts X_test Resulting Shape: {X_test.shape}")
        # print(f"After tts y_train Resulting Shape: {y_train.shape}")
        # print(f"After tts y_test Resulting Shape: {y_test.shape}")
        
        #Make and fit a model
        glm_bin = sm.GLM(
            y_train,
            X_train,
            family=sm.families.Binomial(link=sm.families.links.Logit())).fit()
        
        #Stash results as accessible attributes
        self.results = glm_bin.summary()
        self.model = glm_bin
        self.pvalues = np.round(glm_bin.pvalues, 4).to_frame().sort_values(by = 0)

In [7]:
df2 = pd.read_csv('../datasets/alaska_single_engine_clean.csv')

In [10]:
df2['model'] = df2['model'].str.upper()

In [11]:
df2['make'] = df2['make'].str.upper()

In [15]:
df2['occurred_near_airport'] = 1 - df2['airport_name'].str.contains('Unknown').astype(int)

In [18]:
df2['purpose_of_flight'] = df2['purpose_of_flight'].map(lambda x: 'UNK' if x=='Unknown' else x)

In [19]:
def is_top_model(model, top_x):
    top_list = list(df2['model'].value_counts()[:top_x + 1].index.str.upper())
    return True if model.upper() in top_list else False

In [20]:
df2['model'] = [x.upper() if is_top_model(x,200) else 'UNCOMMON MODEL' for x in df2['model']]

In [21]:
def is_top_make(make, top_x):
    top_list = list(df2['make'].value_counts()[:top_x + 1].index.str.upper())
    return True if make.upper() in top_list else False

In [22]:
df2['make'] = [x.upper() if is_top_make(x,50) else 'UNCOMMON MAKE' for x in df2['make']]


In [38]:
X = df2.drop(['ntsb_no', 'probable_cause', 'airport_name', 'event_type', 'mkey', 'city', 'n', 'has_safety_rec', 'report_type', 'highest_injury_level', 'fatal_injury_count', 'serious_injury_count', 'minor_injury_count', 'airport_id', 'far', 'aircraft_damage', 'operator', 'event_year', 'event_season', 'event_day', 'aircraft_category', 'has_injury', 'event_time','has_aircraft_damage'], axis=1)

In [39]:
tst_X = pd.get_dummies(data= X, columns = ['make', 'model', 'scheduled', 'purpose_of_flight', 'weather_condition', 'event_month', 'event_hour'], drop_first=True)

In [25]:
y = df2['has_injury']

In [115]:
# SAMPLE!
# create a new object that is ultimately a fitted binomial model
binomial_model_1 = aviation_data_model( X, y, [
    # Include tuples of (column name, specific value you want dropped)
    # Just put 'first' if you don't care about dropping a specific value
    ('make', 'CESSNA'),
    ('model', 'PA-18'),
    ('scheduled', 'first'),
    ('purpose_of_flight', 'first'),
    ('weather_condition', 'first'),
    ('event_month', 'first'),
    ('event_hour', 'first')
])

# call the fit_binomial() model to get it!
binomial_model_1.fit_binomial()

# access the results
res = binomial_model_1.results
# or the p values
pvalues = binomial_model_1.pvalues

In [76]:
X_tr, X_test, y_tr, y_test = glm1.train_test(glm2.x_con, y)

In [77]:
X_tr

,const,latitude,longitude,amateur_built,is_accident,occurred_near_airport,make_AERO COMMANDER,make_AERONCA,make_AEROSPATIALE,make_AIRBUS,...,event_hour_14,event_hour_15,event_hour_16,event_hour_17,event_hour_18,event_hour_19,event_hour_20,event_hour_21,event_hour_22,event_hour_23
4125,1.0,0.000000,0.000000,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1780,1.0,58.666667,-156.566665,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3841,1.0,59.509582,-139.709701,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5396,1.0,65.249626,-166.360778,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2687,1.0,58.799732,-156.899932,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148,1.0,64.809890,-147.719314,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
884,1.0,61.950000,-149.216659,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2743,1.0,61.080165,-151.160766,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5657,1.0,0.000000,0.000000,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
        glm_bin = sm.GLM(
            y_tr,
            X_tr,
            family=sm.families.Binomial(link=sm.families.links.Logit())).fit()

In [114]:
glm_bin.pvalues

const            9.998662e-01
latitude         3.282227e-01
longitude        5.170413e-01
amateur_built    5.631090e-02
is_accident      4.956795e-07
                     ...     
event_hour_19    2.158478e-01
event_hour_20    1.510521e-01
event_hour_21    2.160681e-01
event_hour_22    1.957759e-01
event_hour_23    2.404507e-01
Length: 314, dtype: float64